In [1]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

base_model = "codellama/CodeLlama-7b-hf"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")

/home/gcpuser/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.18s/it]


In [2]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "sql-code-llama/checkpoint-160")

In [3]:
def generate_prompt(data_point):
    full_prompt =f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. You must output the SQL query that answers the question.

### Input:
{data_point["question"]}

### Context:
{data_point["context"]}

### Response:
"""
    return full_prompt

In [4]:
import json
with open('spider_create_context_val_db_id.json', 'r') as f:
    data = json.load(f)

# Step 2: Apply the function to each entry
# responses = map(your_function, data)
prompts = []
for data_point in data:
    prompt = generate_prompt(data_point)
    # append prompt to prompts
    prompts.append(prompt)

In [5]:
tokenizer.pad_token = tokenizer.eos_token

In [6]:
# single prompt inference test:
test_prompts = prompts[0:4]
# for prompt in test_prompts:
#     print("prompt is: ", prompt)
tokenized_inputs = tokenizer(test_prompts, return_tensors="pt", padding=True, truncation=True)
# decoded_inputs = #[tokenizer.decode(input, skip_special_tokens=True) for input in tokenized_inputs]
decoded_inputs = [tokenizer.decode(input_ids, skip_special_tokens=True) for input_ids in tokenized_inputs["input_ids"]]
for input in decoded_inputs:
    print(input)
test_prompts == decoded_inputs

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.

### Input:
How many singers do we have?

### Context:
CREATE TABLE singer (Id VARCHAR)

### Response:

You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.

### Input:
Show name, country, age for all singers ordered by age from the oldest to the youngest.

### Context:
CREATE TABLE singer (name VARCHAR, country VARCHAR, age VARCHAR)

### Response:

You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.

### Input:
What is the average, minimum, and maximum age of a

True

In [7]:
batch_outputs = model.generate(**tokenized_inputs, max_new_tokens=100)
# print(batch_outputs)
batch_decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in batch_outputs]
print("decoded batch outputs:")
for output in batch_decoded_outputs:
    print(output)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/gcpuser/miniconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1535: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


decoded batch outputs:
You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.

### Input:
How many singers do we have?

### Context:
CREATE TABLE singer (Id VARCHAR)

### Response:
SELECT COUNT(*) FROM singer

You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.

### Input:
Show name, country, age for all singers ordered by age from the oldest to the youngest.

### Context:
CREATE TABLE singer (name VARCHAR, country VARCHAR, age VARCHAR)

### Response:
SELECT name, country, age FROM singer ORDER BY age DESC

You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must out

In [9]:
batch_size = 12  # You can adjust the batch size based on your GPU capacity
outputs = []

model.eval()
with torch.no_grad(), open('codellama7b-valoutputs2.txt', 'a') as f:
    for i in range(0, len(prompts), batch_size):
        print("i is: ", i)
        batch_inputs = prompts[i:i+batch_size]
        
        # Step 3: Loop over the batches and generate the outputs
        batch_model_inputs = tokenizer(batch_inputs, return_tensors="pt", padding=True, truncation=True)
        
        input_lengths = [len(input_ids) for input_ids in batch_model_inputs["input_ids"]]
        
        batch_outputs = model.generate(**batch_model_inputs, max_new_tokens=100)
        
        # Step 4: Collect the outputs
        batch_decoded_outputs = [tokenizer.decode(output[input_length:], skip_special_tokens=True) for output, input_length in zip(batch_outputs, input_lengths)]
        outputs.extend(batch_decoded_outputs)
        print(outputs)
        for b in batch_decoded_outputs:
            f.write(b.replace("\n", " ") + "\n")
        
# Now, `outputs` is a list containing the outputs for all 900 inputs

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


i is:  0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['SELECT COUNT(*) FROM singer\n', 'SELECT name, country, age FROM singer ORDER BY age DESC\n', "SELECT AVG(age), MIN(age), MAX(age) FROM singer WHERE country = 'France'\n", 'SELECT song_name, song_release_year FROM singer ORDER BY age LIMIT 1\n', 'SELECT DISTINCT country FROM singer WHERE age > 20\n', 'SELECT country, COUNT(*) FROM singer GROUP BY country\n', 'SELECT song_name FROM singer WHERE age > (SELECT AVG(age) FROM singer)\n', 'SELECT LOCATION, name FROM stadium WHERE capacity BETWEEN 5000 AND 10000\n', 'SELECT MAX(average), AVG(capacity) FROM stadium\n', 'SELECT AVG(capacity), MAX(capacity) FROM stadium\n', 'SELECT name, capacity FROM stadium ORDER BY average DESC LIMIT 1\n', 'SELECT COUNT(*) FROM concert WHERE YEAR = 2014 OR YEAR = 2015\n']
i is:  12


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

['SELECT COUNT(*) FROM singer\n', 'SELECT name, country, age FROM singer ORDER BY age DESC\n', "SELECT AVG(age), MIN(age), MAX(age) FROM singer WHERE country = 'France'\n", 'SELECT song_name, song_release_year FROM singer ORDER BY age LIMIT 1\n', 'SELECT DISTINCT country FROM singer WHERE age > 20\n', 'SELECT country, COUNT(*) FROM singer GROUP BY country\n', 'SELECT song_name FROM singer WHERE age > (SELECT AVG(age) FROM singer)\n', 'SELECT LOCATION, name FROM stadium WHERE capacity BETWEEN 5000 AND 10000\n', 'SELECT MAX(average), AVG(capacity) FROM stadium\n', 'SELECT AVG(capacity), MAX(capacity) FROM stadium\n', 'SELECT name, capacity FROM stadium ORDER BY average DESC LIMIT 1\n', 'SELECT COUNT(*) FROM concert WHERE YEAR = 2014 OR YEAR = 2015\n', 'SELECT T1.name, COUNT(*) FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id = T2.stadium_id GROUP BY T1.stadium_id\n', 'SELECT T1.name, T1.capacity FROM stadium AS T1 JOIN concert AS T2 ON T1.stadium_id = T2.stadium_id WHERE T2.year >=

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['SELECT COUNT(*) FROM singer\n', 'SELECT name, country, age FROM singer ORDER BY age DESC\n', "SELECT AVG(age), MIN(age), MAX(age) FROM singer WHERE country = 'France'\n", 'SELECT song_name, song_release_year FROM singer ORDER BY age LIMIT 1\n', 'SELECT DISTINCT country FROM singer WHERE age > 20\n', 'SELECT country, COUNT(*) FROM singer GROUP BY country\n', 'SELECT song_name FROM singer WHERE age > (SELECT AVG(age) FROM singer)\n', 'SELECT LOCATION, name FROM stadium WHERE capacity BETWEEN 5000 AND 10000\n', 'SELECT MAX(average), AVG(capacity) FROM stadium\n', 'SELECT AVG(capacity), MAX(capacity) FROM stadium\n', 'SELECT name, capacity FROM stadium ORDER BY average DESC LIMIT 1\n', 'SELECT COUNT(*) FROM concert WHERE YEAR = 2014 OR YEAR = 2015\n', 'SELECT T1.name, COUNT(*) FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id = T2.stadium_id GROUP BY T1.stadium_id\n', 'SELECT T1.name, T1.capacity FROM stadium AS T1 JOIN concert AS T2 ON T1.stadium_id = T2.stadium_id WHERE T2.year >=

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['SELECT COUNT(*) FROM singer\n', 'SELECT name, country, age FROM singer ORDER BY age DESC\n', "SELECT AVG(age), MIN(age), MAX(age) FROM singer WHERE country = 'France'\n", 'SELECT song_name, song_release_year FROM singer ORDER BY age LIMIT 1\n', 'SELECT DISTINCT country FROM singer WHERE age > 20\n', 'SELECT country, COUNT(*) FROM singer GROUP BY country\n', 'SELECT song_name FROM singer WHERE age > (SELECT AVG(age) FROM singer)\n', 'SELECT LOCATION, name FROM stadium WHERE capacity BETWEEN 5000 AND 10000\n', 'SELECT MAX(average), AVG(capacity) FROM stadium\n', 'SELECT AVG(capacity), MAX(capacity) FROM stadium\n', 'SELECT name, capacity FROM stadium ORDER BY average DESC LIMIT 1\n', 'SELECT COUNT(*) FROM concert WHERE YEAR = 2014 OR YEAR = 2015\n', 'SELECT T1.name, COUNT(*) FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id = T2.stadium_id GROUP BY T1.stadium_id\n', 'SELECT T1.name, T1.capacity FROM stadium AS T1 JOIN concert AS T2 ON T1.stadium_id = T2.stadium_id WHERE T2.year >=